Import dependencies

In [ ]:
import numpy as np
import freud
from ase.io import read
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from tqdm import tqdm

In [ ]:
def get_order(frames, order_parameter_degree, r_max):

    """ returns  the order parameter for a list of frames, 

    returns bins and the kernel density estimate of the order parameter

    Parameters
    ----------
    frames : list of ase.Atoms objects
        List of frames to compute the order parameter for
    order_parameter_degree : int
        The degree of the order parameter to compute
    r_max : float
        Maximum radius to consider for neighbors
        ie. the cutoff chosen in the deepCG paper is 3.4
    
    #IMPORTANT: nans are replaced with 0.0
    effectively removing them from the data
    """

    q_values = []

# Loop through each frame
    for frame in tqdm(frames):
        frame.wrap()
        ql = freud.order.Steinhardt(order_parameter_degree, average=True)

        positions = frame.get_positions()
        box_size = frame.get_cell().lengths()
        box = freud.box.Box(Lx=box_size[0], Ly=box_size[1], Lz=box_size[2])
        
        # Compute the local ql for each particle
        ql.compute((box, positions), {'r_max':r_max})
        q_values.append(ql.particle_order)
    
    N = 100
    bins = np.linspace(
        0.,
        0.5,
        N,
    )

    kde = KernelDensity(bandwidth=0.004)

    vals = np.hstack(q_values).flatten()[:, np.newaxis]

    # from the Freud documentation:
    # =============================
    # The value of per-particle order parameter
    # will be set to NaN for particles with no neighbors.
    # We choose this value rather than setting the order parameter to 0
    # because in more complex order parameter calculations (such as when computing the ),
    # it is possible to observe a value of 0 for the per-particle order parameter
    # even with a finite number of neighbors. 
    # If you would like to ignore this distinction,
    # you can mask the output order parameter 
    # values using NumPy: numpy.nan_to_num(particle_order).
    
    vals = np.nan_to_num(vals, nan=0.0)

    kde.fit(vals)
    ql_proj = np.exp(kde.score_samples(bins[:, np.newaxis]))


    return ql_proj, bins

Load frames

In [ ]:
frames_0_50 = read("/scratch/chong/section5_results/50_0/test.xyz", ":")
frames_1_50 = read("/scratch/chong/section5_results/50_1/test.xyz", ":")
frames_2_50 = read("/scratch/chong/section5_results/50_2/test.xyz", ":")
frames_3_50 = read("/scratch/chong/section5_results/50_3/test.xyz", ":")

frames_0_100 = read("/scratch/chong/section5_results/100_0/test.xyz", ":")
frames_1_100 = read("/scratch/chong/section5_results/100_1/test.xyz", ":")
frames_2_100 = read("/scratch/chong/section5_results/100_2/test.xyz", ":")
frames_3_100 = read("/scratch/chong/section5_results/100_3/test.xyz", ":")

frames_0_1k = read("/scratch/chong/section5_results/1k_0/test.xyz", ":")
frames_1_1k = read("/scratch/chong/section5_results/1k_1/test_new.xyz", ":")
frames_2_1k = read("/scratch/chong/section5_results/1k_2/test.xyz", ":")
frames_3_1k = read("/scratch/chong/section5_results/1k_3/test.xyz", ":")

frames_0_10k = read("/scratch/chong/section5_results/10k_0/test.xyz", ":")
frames_1_10k = read("/scratch/chong/section5_results/10k_1/test.xyz", ":")
frames_2_10k = read("/scratch/chong/section5_results/10k_2/test.xyz", ":")
frames_3_10k = read("/scratch/chong/section5_results/10k_3/test.xyz", ":")


Compute order parameters

In [ ]:
orig_frames = read('/home/chong/CG_water.xyz', index=':')
ql4_orig, bins = get_order(orig_frames, 4, 3.6)
ql6_orig, bins = get_order(orig_frames, 6, 3.6)

ql4_50_list = []
ql6_50_list = []
for frames in [frames_0_50, frames_1_50, frames_2_50, frames_3_50]:
    ql4, bins = get_order(frames, 4, 3.6)
    ql6, bins = get_order(frames, 6, 3.6)
    ql4_50_list.append(ql4)
    ql6_50_list.append(ql6)

ql4_100_list = []
ql6_100_list = []
for frames in [frames_0_100, frames_1_100, frames_2_100, frames_3_100]:
    ql4, bins = get_order(frames, 4, 3.6)
    ql6, bins = get_order(frames, 6, 3.6)
    ql4_100_list.append(ql4)
    ql6_100_list.append(ql6)
    
ql4_1k_list = []
ql6_1k_list = []
for frames in [frames_0_1k, frames_1_1k, frames_2_1k, frames_3_1k]:
    ql4, bins = get_order(frames, 4, 3.6)
    ql6, bins = get_order(frames, 6, 3.6)
    ql4_1k_list.append(ql4)
    ql6_1k_list.append(ql6)

ql4_10k_list = []
ql6_10k_list = []
for frames in [frames_0_10k, frames_1_10k, frames_2_10k, frames_3_10k]:
    ql4, bins = get_order(frames, 4, 3.6)
    ql6, bins = get_order(frames, 6, 3.6)
    ql4_10k_list.append(ql4)
    ql6_10k_list.append(ql6)

In [ ]:
np.save("steinhardt_results/ql4_50.npy", np.array(ql4_50_list))
np.save("steinhardt_results/ql4_100.npy", np.array(ql4_100_list))
np.save("steinhardt_results/ql4_1k.npy", np.array(ql4_1k_list))
np.save("steinhardt_results/ql4_10k.npy", np.array(ql4_10k_list))

In [ ]:
np.save("steinhardt_results/ql6_50.npy", np.array(ql6_50_list))
np.save("steinhardt_results/ql6_100.npy", np.array(ql6_100_list))
np.save("steinhardt_results/ql6_1k.npy", np.array(ql6_1k_list))
np.save("steinhardt_results/ql6_10k.npy", np.array(ql6_10k_list))